### In this lab, we will build a deep research utility. You can give a query to this utility. It will perform "x" number of web searches to research the query and then
### we finally create a nice report from it and send it by email.

In [1]:
from dotenv import load_dotenv
import os
import sendgrid
from sendgrid.helpers.mail import Email, To, Content, Mail
from agents import Agent, Runner, trace, function_tool, input_guardrail, output_guardrail, WebSearchTool, ComputerTool, FileSearchTool, ModelSettings
import asyncio
from pydantic import BaseModel, Field
from typing import Dict
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)

In [3]:
# OpenAI agents SDK provides Hosted tools. i.e. tools which are available out of the box for you to use.
# WebSearchTool --> lets an agent search the web (we will use this in this utility)
# FileSearchTool --> allows retrieving information from your OpenAI vector stores (we will see this later)
# ComputerTool --> allows automating computer use tasks like taking screenshots and clicking.

In [3]:
# Lets create a basic web search Agent. This agent is given a search term (eg: "Climate Change implications") and it can go out and perform a web search
# and come back with a lot of text on that topic.
SEARCH_INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(name="SearchAgent",
                     instructions=SEARCH_INSTRUCTIONS,
                     model='gpt-4o-mini',
                     tools=[WebSearchTool(search_context_size='low')], # keep it low to limit your OpenAI billing. WebSearchTool is expensive.
                     model_settings=ModelSettings(tool_choice='required')
                    )

In [ ]:
# Lets test it out.
message = "What are the implications of Climate Change?"
with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))


In [4]:
### Next, we will create a planner_agent with some structured output. We expect the planner agent to return "x" number of search terms for a given query.
### We also want the agent to return the reasoning for the search term. i.e. why these search terms were chosen. What is the importance of these search terms.
SEARCH_COUNT = 3

class WebSearchTerm(BaseModel):
    reason: str = Field(description="Your reasoning for why this search term is important to the query?")
    term: str = Field(description="The actual search term you will use to search the web.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchTerm] = Field(description="A list of web searches to perform to best answer the query.")

PLANNER_INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {SEARCH_COUNT} terms to query for."

planner_agent = Agent(name="PlannerAgent",
                      instructions=PLANNER_INSTRUCTIONS,
                      model='gpt-4o-mini',
                      output_type=WebSearchPlan,
                      )

In [ ]:
# lets test it out.
message = "Implications of Climate Change?"
with trace("Planner"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

In [5]:
# Now lets create some helper function to perform the web searches, prepare the report and send it by email.
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """
    Send an email with the given subject and body.
    """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("rajat.girotra@gmail.com")
    to_email = To("rajatgirotra@yahoo.com")
    subject = subject
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content)
    response = sg.client.mail.send.post(request_body=mail.get())
    return {'status': 'success'}

In [ ]:
send_email("Test Email", "<h1>Hello, World!</h1>")

In [7]:
# lets create an agent for sending emails.
EMAIL_INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(name="EmailAgent",
                    instructions=EMAIL_INSTRUCTIONS,
                    model='gpt-4o-mini',
                    tools=[send_email],
                    model_settings=ModelSettings(tool_choice='required')
                    )


In [8]:
# Finally a writer agent that will collate all the responses from the planner and search agent. It will formulate a big research report and send it by email.
WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [ ]:
type(WRITER_INSTRUCTIONS)

In [10]:
# We do not build a workflow here. Rather we manually call each of the agents and pass the results to the next agent.

# plan_searches will call the PlannerAgent
async def plan_searches(query: str) -> WebSearchPlan:
    """ Use the planner_agent to plan which searches to run for the query."""
    results = await Runner.run(planner_agent, input=query)
    print(f"Will perform {len(results.final_output.searches)} searches.")
    return results.final_output


async def perform_searches(search_term: WebSearchPlan):
    """ Call search() for each item in the search_terms list."""
    print(f"searching...")
    tasks = [asyncio.create_task(search(search_item)) for search_item in search_term.searches]
    results = await asyncio.gather(*tasks)
    print(f"finished searching.")
    return results

# search will call the SearchAgent to perform a single web search.
async def search(search_item: WebSearchTerm):
    """ Use the search agent to run a single web search."""
    # create input message for the SearchAgent
    input = f"Search term: {search_item.term}\nReason for searching: {search_item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [11]:
# These two functions will create a detailed report using the WriterAgent and send email using the EmailAgent.
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")
